In [ ]:
# import libraries
import sys
import os
import yaml
import h5py
import pandas as pd
import flatdict as fd
import matplotlib.pylab as plt
from jupyterlab_h5web import H5Web
# import modules with the functionalities offered by CompositionSpace
from compositionspace.utils import get_file_size
# from compositionspace.io import get_reconstructed_positions, get_iontypes, get_ranging_info
from compositionspace.preparation import ProcessPreparation
from compositionspace.autophase import ProcessAutomatedPhaseAssignment
from compositionspace.segmentation import ProcessSegmentation
from compositionspace.clustering import ProcessClustering

In [ ]:
# ! pip list
MY_PROCESSED_DATA_PATH = f"{os.getcwd()}"
print(f"Executing compositionspace in the following working directory: {os.getcwd()}")

## Load reconstruction and ranging and voxelize with rectangular transfer function without creating slices

In [ ]:
workdir = "/home/kaiobach/Research/paraprobe-toolbox/teaching/example_analyses/iuc09_saksena"
workdir = "/home/kaiobach/Research/paraprobe-toolbox/teaching/example_analyses/usa_denton_smith"
simid = 1
RECONSTRUCTION_AND_RANGING = (f"{workdir}/..")
RECONSTRUCTION_AND_RANGING = (f"{workdir}/PARAPROBE.Transcoder.Results.SimID.1.nxs",
                              f"{workdir}/PARAPROBE.Ranger.Results.SimID.1.nxs")
config_file_path = f"{MY_PROCESSED_DATA_PATH}/tests/experiment_params.yaml"
results_file_path = f"{MY_PROCESSED_DATA_PATH}/CompositionSpace.Results.{simid}.nxs"

In [ ]:
get_file_size(RECONSTRUCTION_AND_RANGING[0])
get_file_size(RECONSTRUCTION_AND_RANGING[1])
# H5Web(RECONSTRUCTION_AND_RANGING[0])
# H5Web(RECONSTRUCTION_AND_RANGING[1])

In [ ]:
voxelize = ProcessPreparation(config_file_path, results_file_path, entry_id=1, verbose=True)
voxelize.run(recon_file_path=RECONSTRUCTION_AND_RANGING[0],
             range_file_path=RECONSTRUCTION_AND_RANGING[1])

In [ ]:
get_file_size(results_file_path)
# H5Web(results_file_path)

Voxelization is performed on elements not on iontypes, i.e. using a atomic decomposition!

## Automated phase assignment

In [ ]:
autophase = ProcessAutomatedPhaseAssignment(config_file_path, results_file_path, entry_id=1, verbose=True)
autophase.run()

In [ ]:
get_file_size(results_file_path)
# H5Web(results_file_path)

## Segmentation PCA and IC minimization

In [ ]:
segmentation = ProcessSegmentation(config_file_path, results_file_path, entry_id=1, verbose=True)
segmentation.run()

In [ ]:
get_file_size(results_file_path)
# H5Web(results_file_path)

## DBScan clustering

In [ ]:
clustering = ProcessClustering(config_file_path, results_file_path, entry_id=1, verbose=True)
clustering.run_and_write_results()

In [ ]:
get_file_size(results_file_path)
H5Web(results_file_path)

## Meshing

Test for now with the SiGe dataset.

In [ ]:
sige_file_path = "Output_DBSCAN_segmentation_phase_1.h5"
H5Web(sige_file_path)

In [ ]:
import h5py
import numpy as np
with h5py.File(sige_file_path, "r") as h5r:
    n_vxls = 0
    aabb3d = np.zeros((3, 2), np.float64)
    for dim in [0, 1, 2]:
        aabb3d[dim, :] = [np.finfo(np.float64).max, np.finfo(np.float64).min]
    # print(aabb3d)
    for key in h5r["1"].keys():
        for dim in [0, 1, 2]:
            mimx = (np.min(h5r["1"][key][:, dim]), np.max(h5r["1"][key][:, dim]))
            if mimx[0] <= aabb3d[dim, 0]:
                aabb3d[dim, 0] = mimx[0]
            if mimx[1] >= aabb3d[dim, 1]:
                aabb3d[dim, 1] = mimx[1]
            n_vxls += int(np.shape(h5r["1"][key])[0])
    print(aabb3d)
    print(n_vxls)
    # assume cubic vxl 2nm edge length

Assume that the data were discretized on the following rectangular grid with 2nm cubic voxel

In [ ]:
nx = int((88--96)/2)
ny = int((94--96)/2)
nz = int((0--222)/2)
grid = np.zeros((nx, ny, nz), np.uint32)
# that grid should intentionally be a cuboid to enable checking correct dimensions

def i_to_xyz(i):
    z = int(i / (nx * ny))
    rem = i - (nx * ny * z)
    y = int(rem / nx)
    x = rem - (y * nx)
    return (x, y, z)

with h5py.File(sige_file_path, "r") as h5r:
    for key in h5r["1"].keys():
        jds = np.asarray(h5r["1"][key][:, 3], np.uint32)
        for j in jds:
            x, y, z = i_to_xyz(j)
            grid[x, y, z] = int(key) + 1
        print(key)
print(np.shape(grid))
print(np.unique(grid))

In [ ]:
import h5py
with h5py.File("input.grid.nxs", "w") as h5w:
    h5w.create_dataset("/grid", compression="gzip", compression_opts=1, data=grid)

In [ ]:
H5Web("input.grid.nxs")